In [1]:
from __future__ import print_function 
import numpy as np
import pandas as pd
import tensorflow as tf 

In [2]:
label_num = 8

In [3]:
f = np.load('corpus_all_47001.npz')

In [4]:
f.files

['train_doc', 'valid_doc', 'test_doc']

In [5]:
train_doc = f['train_doc']

In [6]:
print(type(train_doc))

<class 'numpy.ndarray'>


In [7]:
print(train_doc[0][1])

0


In [8]:
valid_doc = f['valid_doc']

In [9]:
test_doc = f['test_doc']

In [10]:
min_count = 100
max_count = 0

for doc in train_doc:
    for sent in doc[0]:
        if len(sent) < min_count:
            min_count = len(sent)
        if len(sent) > max_count:
            max_count = len(sent)
            
print(min_count)
print(max_count)


1
290


In [11]:
def convert2words(doc):
    return [word for sent in doc for word in sent]

In [12]:
train_doc_words = np.asarray([(convert2words(doc[0]), doc[1]) for doc in train_doc])

In [13]:
# print(train_doc_words[0][0])

In [14]:
# print(train_doc[0])

## Build the vocabulary

In [15]:
from collections import Counter 

In [16]:
counts_new = []
            
words_count_list = Counter([word for doc in train_doc for sent in doc[0] for word in sent]).most_common(47001)

In [17]:
print(len(words_count_list))
print(words_count_list[:10])

47001
[('the', 148314), ('and', 106841), ('to', 90699), ('of', 81748), ('a', 75296), ('that', 67989), ('i', 58165), ('in', 56239), ('it', 51467), ('we', 49525)]


In [18]:
l = [1,2,3,4,4]
for i in enumerate(l, 1):
    print(i)

(1, 1)
(2, 2)
(3, 3)
(4, 4)
(5, 4)


In [19]:
word2index = {item[0]: index for index, item in enumerate(words_count_list, 1)}
index2word = {index: item[0] for index, item in enumerate(words_count_list, 1)}

In [20]:
print(word2index['and'])
print(index2word[2])

2
and


In [21]:
def doc2index(doc, word2index):
    return [word2index[word] for word in doc]

In [22]:
train_wordasindex = np.asarray([doc2index(doc[0], word2index) for doc in train_doc_words])

In [23]:
print(train_wordasindex.shape)
print(train_wordasindex[0][0])
# print(train_wordasindex[0])

(1579,)
72


In [24]:
# API for corpuse format conversion 

def convert2words(doc):
    return [word for sent in doc for word in sent]

def doc2index(doc, word2index):
    return [[word2index[word]] for word in doc]

def convert_corpus(corpus, word2index):
    corpus_words = [(convert2words(doc[0]), doc[1]) for doc in corpus]
    return np.asarray([doc2index(doc[0], word2index) for doc in corpus_words])

In [25]:
train_wordasindex = convert_corpus(train_doc, word2index)
valid_wordasindex = convert_corpus(valid_doc, word2index)
test_wordasindex = convert_corpus(test_doc, word2index)

In [26]:
print(train_wordasindex.shape)
print(len(train_wordasindex[0]))
print(train_wordasindex[0][0])

(1579,)
1364
[72]


## Embed words 

In [27]:
from gensim.models import Word2Vec

In [28]:
model = Word2Vec.load("word2vec_model_vocab47001_mincount1")

In [29]:
print(len(model.wv.vocab))

47001


In [30]:
print(word2index['computer'])
print(index2word[394])
# print(model.wv[index2word[394]])
print(model.wv[index2word[394]].shape)
# model.most_similar(index2word[394])

393
child
(100,)


In [31]:
def embed_from_index(model, idx):
    if idx != 0:
        return model.wv[index2word[idx]]
    else:
        return np.zeros(shape=model.wv[index2word[1]].shape)

def embed_corpus(corpus, word2index, model):
    temp = list(corpus)
    return np.asarray([[embed_from_index(model, idx) for sent in doc for idx in sent] for doc in temp])

In [32]:
# train_wordasindex_embed = embed_corpus(train_doc, word2index, model)


In [33]:
# print(train_wordasindex_embed.shape)
# print(len(train_wordasindex_embed[0]))
# print(train_wordasindex_embed[0][0])

In [34]:
# print(train_wordasindex_embed[0][0].shape)

In [35]:
# train_wordasindex_embed = embed_corpus(train_doc, word2index, model)
# valid_wordasindex_embed = embed_corpus(valid_doc, word2index, model)
# test_wordasindex_embed = embed_corpus(test_doc, word2index, model)

## Convert labels 

In [36]:
def encode_label(label, size):
    l = [0]*size
    l[label] = 1
    return l

def encode_class(corpus, size):
    return np.asarray([encode_label(doc[1], size) for doc in corpus])

In [37]:
train_label = encode_class(train_doc, 8)
valid_label = encode_class(valid_doc, 8)
test_label = encode_class(test_doc, 8)

In [38]:
print(train_label.shape)

(1579, 8)


In [101]:
label_tally = {i:0 for i in range(8)}

for doc in train_doc:
    label_tally[doc[1]] += 1
    
for key in label_tally:
    print(key, label_tally[key])

0 966
1 275
2 97
3 112
4 18
5 85
6 10
7 16


In [39]:
train_data = (train_wordasindex, train_label)

In [40]:
print(train_data[0].shape)
print(train_data[1].shape)
# print(train_data[0][0])

(1579,)
(1579, 8)


In [41]:
min_count = 100
max_count = 0

for doc in valid_wordasindex:
    if len(doc) < min_count:
        min_count = len(doc)
    if len(doc) > max_count:
        max_count = len(doc)

print(min_count)
print(max_count)


16
6686


## Classification model 

In [42]:
max_features = 47001

learning_rate = 0.001
maxlen = 6700
batch_size = 50
total_batch = int(train_data[0].shape[0]/batch_size)
input_dims = 100
num_hidden= 50
epochs = 100

index=0

In [43]:
print(total_batch)

31


### Padding with 0s

In [44]:
train_data
train_wordasindex
valid_wordasindex
test_wordasindex
train_label
valid_label
test_label
print(train_wordasindex.shape)
# print(train_wordasindex[0])

(1579,)


In [45]:
for sent in train_wordasindex:
    for word_index in sent:
        if word_index == 0:
            print("failed")
            

In [46]:
[[0]]*5

[[0], [0], [0], [0], [0]]

In [47]:
def pad_with_zeros(sequence, maxlen):
    if len(sequence) > maxlen:
        raise Exception
    else:
        return sequence+[[0]]*(maxlen-len(sequence))

In [48]:
k = [1, 2, 3]
pad_with_zeros(k,5)

[1, 2, 3, [0], [0]]

In [49]:
def corpus_pad_with_zeros(corpus, maxlen):
    return np.asarray([pad_with_zeros(sent, maxlen) for sent in corpus])

In [50]:
train_corpus = corpus_pad_with_zeros(train_wordasindex, maxlen)
valid_corpus = corpus_pad_with_zeros(valid_wordasindex, maxlen)
# test_corpus = corpus_pad_with_zeros(test_wordasindex, maxlen)



In [51]:
# print(train_corpus.shape)
# print(train_label.shape)
# print(train_corpus[0].shape)
# print(train_corpus[3].shape)
# print(train_corpus[45].shape)
# print(train_corpus[90].shape)
# print(train_corpus[0])

In [52]:
train_corpus_embed = embed_corpus(train_corpus, word2index, model)
valid_corpus_embed = embed_corpus(valid_corpus, word2index, model)
# test_corpus_embed = embed_corpus(test_corpus, word2index, model)

In [102]:
test_corpus = corpus_pad_with_zeros(test_wordasindex, maxlen)

In [103]:
test_corpus_embed = embed_corpus(test_corpus, word2index, model)

In [104]:
np.savez('corpus_embeded_all_47001', train_corpus_embed=train_corpus_embed, valid_corpus_embed=valid_corpus_embed, test_corpus_embed=test_corpus_embed)

In [53]:
# print(train_corpus_embed.shape)
# print(train_label.shape)
# print(train_corpus_embed[0].shape)
# print(train_corpus_embed[3].shape)
# print(train_corpus_embed[45].shape)
# print(train_corpus_embed[90].shape)
# print(train_corpus_embed[0])

In [54]:
print(valid_corpus.shape)
print(valid_label.shape)

(250, 6700, 1)
(250, 8)


In [55]:
def next_batch(data, index, size):
    """ return next batch in format: index, x batch, y batch
    """
    if index + size <= data[0].shape[0]:
        return index+size, data[0][index:index+size], data[1][index:index+size]
    else:
        return index+size-data[0].shape[0], np.concatenate((data[0][index:],data[0][:index+size-data[0].shape[0]]), 0), \
    np.concatenate((data[1][index:],data[1][:index+size-data[1].shape[0]]), 0)

In [56]:
def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length

In [57]:
# tensorflow wants a data format of [Batch Size, Sequence Length, Input Dimension].
x = tf.placeholder(tf.float32, [None, maxlen, input_dims])
y = tf.placeholder(tf.float32, [None, 8])

In [110]:
comp = tf.placeholder(tf.float32, [None])

In [58]:
with tf.variable_scope('cellsdef'):
    cell_fw = tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)
    cell_bw = tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)

In [59]:
with tf.variable_scope('lstm_rnn_k'):
    val, state = tf.nn.dynamic_rnn(cell_fw, x, dtype=tf.float32, sequence_length=length(x))    # val are the h_ts 

In [60]:
print(val.shape)
print(len(state))
print(state[0].shape)
print(state[1].shape)

(?, 6700, 50)
2
(?, 50)
(?, 50)


In [61]:
print(length(x))

Tensor("Cast:0", shape=(?,), dtype=int32)


In [91]:
W = tf.Variable(tf.truncated_normal(shape=[50, 32]))
b = tf.Variable(tf.constant(0.0, shape=[32]))

# W2 = tf.Variable(tf.truncated_normal(shape=[32, 16]))
# b2 = tf.Variable(tf.constant(0.0, shape=[16]))


V = tf.Variable(tf.truncated_normal(shape=[32, 8]))
c = tf.Variable(tf.constant(0.0, shape=[8]))

dropout_rate = tf.placeholder(tf.float32)

h = tf.nn.elu(tf.matmul(tf.reduce_mean(val, axis=1), W) + b)
# h_drop = tf.nn.dropout(h, keep_prob=dropout_rate)
# h2 = tf.nn.elu(tf.matmul(h_drop, W2) + b2)

# h = tf.nn.relu(tf.matmul(state[1], W) + b)
u = tf.matmul(h, V) + c
p = tf.nn.softmax(u)
pred = tf.argmax(p, 1)
loss = tf.reduce_mean(tf.reduce_sum(-tf.cast(y, tf.float32)*tf.log(p), 1))

In [92]:
learning_rate = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [93]:
# using HK@k??
print(p.shape)
a, b = tf.nn.top_k(p, k=3)
print(a.shape, b.shape)
print(tf.argmax(y, 1).shape)
t = tf.unstack(b, axis=1)
print(len(t))
print(t[0].shape)

(?, 8)
(?, 3) (?, 3)
(?,)
3
(?,)


In [94]:
values, indices = tf.nn.top_k(p, k=3)
y_truth = tf.cast(tf.argmax(y, 1), tf.int32)
top_list = tf.unstack(indices, axis=1)
hit_rate = tf.reduce_mean(tf.cast(tf.equal(top_list[0], y_truth), tf.float32)\
    + tf.cast(tf.equal(top_list[1], y_truth), tf.float32)\
    + tf.cast(tf.equal(top_list[2], y_truth), tf.float32))

In [95]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, tf.argmax(y, 1)), tf.float32))

In [112]:
accuracy_most_likely_class = tf.reduce_mean(tf.cast(tf.equal(pred, tf.cast(comp, tf.int64)), tf.float32))

In [96]:
sess = tf.InteractiveSession()

In [97]:
init = tf.global_variables_initializer()
sess.run(init)

In [113]:
most_class = np.zeros([250])
print(most_class.shape)

(250,)


In [114]:
epochs = 300
for i in range(epochs+1):
    xloss = 0
    acc = 0.0
    
    for j in range(total_batch):
        # need to incoporate y in the batches and expand to 8 classes 
        index, x_, y_ = next_batch((train_corpus_embed, train_label), index, batch_size)
        _, xloss = sess.run([optimizer, loss], feed_dict={x: x_, y: y_, dropout_rate: 0.5})
        
        if j % 10 == 0:
            print("epoch %d, run %d, loss %g" % (i, j, xloss))
            
    if i % 1 == 0:
        acc, hr, acc_most = sess.run([accuracy, hit_rate, accuracy_most_likely_class], feed_dict={x:valid_corpus_embed, y:valid_label, dropout_rate: 1.0, comp: most_class})
        print("epoch %d, Validation acc: %g" % (i, acc * 100), end="")
        print("%", end="")
        print(", hit rate: %g" % (hr *100), end="")
        print("%", end="")
        print(", class 0 accuracy: %g" % (acc_most *100), end="")
        print("%")

epoch 0, run 0, loss 0.663523
epoch 0, run 10, loss 0.749787
epoch 0, run 20, loss 1.28233
epoch 0, run 30, loss 0.527809
epoch 0, Validation acc: 45.2%, hit rate: 78.4%, class 0 accuracy: 78.4%
epoch 1, run 0, loss 0.759069
epoch 1, run 10, loss 0.755754
epoch 1, run 20, loss 1.25488
epoch 1, run 30, loss 0.488679
epoch 1, Validation acc: 46%, hit rate: 78.8%, class 0 accuracy: 72.4%
epoch 2, run 0, loss 0.657462
epoch 2, run 10, loss 1.01113
epoch 2, run 20, loss 1.13904
epoch 2, run 30, loss 0.566496
epoch 2, Validation acc: 46.4%, hit rate: 79.6%, class 0 accuracy: 74%
epoch 3, run 0, loss 0.530997
epoch 3, run 10, loss 0.722613
epoch 3, run 20, loss 1.04754
epoch 3, run 30, loss 0.881299
epoch 3, Validation acc: 46.8%, hit rate: 78.8%, class 0 accuracy: 69.6%
epoch 4, run 0, loss 0.530286
epoch 4, run 10, loss 0.720508
epoch 4, run 20, loss 1.20403
epoch 4, run 30, loss 0.943459
epoch 4, Validation acc: 46%, hit rate: 78%, class 0 accuracy: 74%
epoch 5, run 0, loss 0.600844
epoch 

KeyboardInterrupt: 

## Questions

- What are the benefits and downsides of the RNN-based representation over the bag of words representation used last week? 
- How would availability of data affect your answer?
- One possible architectural variant is to use only the final hidden state of the RNN as the document representation (i.e., x) rather than the average of the hidden states over time. How does this work? What are the potential benefits and downsides to this representation?
- Try different RNN architectures, e.g., simple Elman RNNs or GRUs or LSTMs. Which ones work best?
- What happens if you use a bidirectional LSTM (i.e., the dashed arrows in the figure)?